# Setup and Validation

This notebook validates your environment and configuration before running the SLAVV comparison pipeline. It replaces the legacy `validate_setup.py` script.

## ⚠️ Environment Check

**IMPORTANT**: Before proceeding, ensure you are using a properly configured virtual environment.

### Quick Check
Run the cell below. If you see errors, follow the setup guide at `docs/guides/SETUP.md`.

In [6]:
import sys
import os
from pathlib import Path

print(f"Python Executable: {sys.executable}")
print(f"Python Version: {sys.version}")

# Check if we're in a virtual environment
def is_venv():
    # Standard venv/virtualenv check
    if hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix):
        return True
    # Conda check via environment variable
    if os.environ.get('CONDA_DEFAULT_ENV') and os.environ.get('CONDA_DEFAULT_ENV') != 'base':
        return True
    # Path check: environment usually resides in 'envs' directory for conda
    if 'envs' in sys.executable:
        return True
    return False

in_venv = is_venv()

if in_venv:
    print("✅ Running in a virtual environment")
else:
    print("⚠️ NOT running in a virtual environment (or using base conda env)")
    print("   Recommendation: Create a dedicated venv or conda environment. See docs/guides/SETUP.md")

# Try importing the package
try:
    import slavv
    print(f"✅ SLAVV package found at: {slavv.__file__}")
except ImportError:
    print("❌ SLAVV package not installed!")
    print("   Run: pip install -e .")
    print("   See docs/guides/SETUP.md for full setup instructions")

Python Executable: c:\Users\alw4834\.conda\envs\slavv-env\python.exe
Python Version: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
✅ Running in a virtual environment
✅ SLAVV package found at: C:\Users\alw4834\OneDrive - The University of Texas at Austin\Documents 1\GitHub\slavv2python\source\slavv\__init__.py


In [7]:
%load_ext autoreload
%autoreload 2

import subprocess
import sys
from pathlib import Path

# Import validation module directly from installed package
from slavv.dev.validation import Validator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration

Set your paths below. If you are unsure, the defaults usually work for standard installations.

In [8]:
# Path to MATLAB executable
# Windows example: "C:\\Program Files\\MATLAB\\R2023a\\bin\\matlab.exe"
import shutil
matlab_path = shutil.which('matlab') or "C:\\Program Files\\MATLAB\\R2019a\\bin\\matlab.exe"
if not shutil.which('matlab') and matlab_path == "C:\\Program Files\\MATLAB\\R2019a\\bin\\matlab.exe":
    print("⚠️ MATLAB not found in PATH. Using default fallback path. Please update 'matlab_path' below if incorrect.")

# Path to test data volume
# Using data from tests/data directory
project_root = Path(slavv.__file__).parent.parent.parent
test_data = project_root / "tests" / "data" / "slavv_test_volume.tif"

print(f"Project Root: {project_root}")
print(f"MATLAB Path: {matlab_path}")
print(f"Test Data: {test_data}")

Project Root: C:\Users\alw4834\OneDrive - The University of Texas at Austin\Documents 1\GitHub\slavv2python
MATLAB Path: C:\Program Files\MATLAB\R2019a\bin\matlab.EXE
Test Data: C:\Users\alw4834\OneDrive - The University of Texas at Austin\Documents 1\GitHub\slavv2python\tests\data\slavv_test_volume.tif


## Run Validation

Execute the validation checks. This looks for:
1. Required Python packages
2. Valid MATLAB installation
3. Necessary external submodules (Vectorization-Public)
4. Test data integrity

In [9]:
validator = Validator(project_root, matlab_path=matlab_path, test_data_path=str(test_data))

# Run standard checks
validator.check_python_imports()
validator.check_vectorization_public()
validator.check_matlab_executable()
validator.check_test_data()
validator.check_disk_space(project_root / "comparison_output")

# Run specific MATLAB version check (requires launching MATLAB, may take a moment)
print("Checking MATLAB version...")
validator.check_matlab_version()

Checking MATLAB version...


True

## MATLAB Functional Test

This runs a small MATLAB script to confirm it can execute code and find the vectorization library.

In [10]:
# Run the robust functional test from the Validator class
print("Running MATLAB functional test...")
validator.check_matlab_functional()

Running MATLAB functional test...


True

## Summary Results

In [11]:
print("Passed Checks:")
for p in validator.passed:
    print(f"✅ {p.message}")

if validator.warnings:
    print("\nWarnings:")
    for w in validator.warnings:
        print(f"⚠️ {w.message}")

if validator.errors:
    print("\nERRORS:")
    for e in validator.errors:
        print(f"❌ {e.message}")
    print("\n💡 If you see import errors, check docs/guides/SETUP.md")
else:
    print("\n🎉 System is ready for comparison runs!")

Passed Checks:
✅ Python package NumPy is installed
✅ Python package SciPy is installed
✅ Python package tifffile is installed
✅ Python package Matplotlib is installed
✅ SLAVVProcessor can be imported
✅ load_tiff_volume can be imported
✅ Vectorization-Public directory: C:\Users\alw4834\OneDrive - The University of Texas at Austin\Documents 1\GitHub\slavv2python\external\Vectorization-Public
✅ vectorize_V200.m: C:\Users\alw4834\OneDrive - The University of Texas at Austin\Documents 1\GitHub\slavv2python\external\Vectorization-Public\vectorize_V200.m
✅ Vectorization-Public/source directory found
✅ MATLAB executable found: C:\Program Files\MATLAB\R2019a\bin\matlab.EXE
✅ Test data file exists: C:\Users\alw4834\OneDrive - The University of Texas at Austin\Documents 1\GitHub\slavv2python\tests\data\slavv_test_volume.tif
✅ Test data loaded successfully: shape=(222, 512, 512), dtype=int16
✅ Sufficient disk space: 27.7 GB available
✅ MATLAB -batch flag is supported (R2019a or later)
✅ MATLAB fun